In [44]:
from fredapi import Fred
import pandas as pd
# import glob
import pymysql

In [45]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [46]:
api_key = os.getenv("fred_api_key")

In [47]:
# api_key = "ae5480bbc5e84829fa0b4bf032c59e32"

In [48]:
# csv_files = glob.glob("DATA/*.csv")

In [49]:
# csv_files

In [50]:
# fred_tickers = [csv.replace("DATA\\", "").replace(".csv", "").replace("FRED_", "") for csv in csv_files if "FRED" in csv ]

In [51]:
fred_tickers = ['DCOILWTICO', 'DEXCHUS', 'DEXJPUS', 'DEXUSEU', 'DGS10', 'DTWEXBGS', 'SP500']

In [52]:
fred = Fred(api_key=api_key)

In [53]:
host = os.getenv("db_host")
user = os.getenv("db_user")
password = os.getenv("db_password")
database = os.getenv("db_database")

In [54]:
# DB 연결 설정
conn = pymysql.connect(
    host=host,
    user=user,
    password=password,
    database=database,
    charset='utf8mb4',
    autocommit=True
)


In [55]:
def update_or_insert_db(df, conn):
    with conn.cursor() as cursor:
        for _, row in df.iterrows():
            # 먼저 존재 여부 확인
            check_sql = """
                SELECT COUNT(*) FROM FRED
                WHERE date = %s AND item = %s
            """
            cursor.execute(check_sql, (row['date'], row['item']))
            count = cursor.fetchone()[0]

            if count > 0:
                # 존재하면 UPDATE
                update_sql = """
                    UPDATE FRED
                    SET value = %s
                    WHERE date = %s AND item = %s
                """
                cursor.execute(update_sql, (row['value'], row['date'], row['item']))
                print("update", row['value'], row['date'], row['item'])
            else:
                # 존재하지 않으면 INSERT
                insert_sql = """
                    INSERT INTO FRED (date, value, item)
                    VALUES (%s, %s, %s)
                """
                cursor.execute(insert_sql, (row['date'], row['value'], row['item']))
                print("insert", row['value'], row['date'], row['item'])


In [56]:
for fred_ticker in fred_tickers:
    # FRED API를 통해 데이터 가져오기
    fdata = fred.get_series(observation_start="2025-06-01", series_id=fred_ticker)
    df = pd.DataFrame({"date": fdata.index, "value": fdata.values, "item": fred_ticker})
    update_or_insert_db(df, conn)
    # print(df.tail())

update 63.27 2025-06-02 00:00:00 DCOILWTICO
update 64.1 2025-06-03 00:00:00 DCOILWTICO
update 63.57 2025-06-04 00:00:00 DCOILWTICO
update 64.06 2025-06-05 00:00:00 DCOILWTICO
update 65.3 2025-06-06 00:00:00 DCOILWTICO
update 65.99 2025-06-09 00:00:00 DCOILWTICO
update 7.1975 2025-06-02 00:00:00 DEXCHUS
update 7.1872 2025-06-03 00:00:00 DEXCHUS
update 7.1844 2025-06-04 00:00:00 DEXCHUS
update 7.175 2025-06-05 00:00:00 DEXCHUS
update 7.1886 2025-06-06 00:00:00 DEXCHUS
update 142.76 2025-06-02 00:00:00 DEXJPUS
update 143.93 2025-06-03 00:00:00 DEXJPUS
update 142.91 2025-06-04 00:00:00 DEXJPUS
update 143.87 2025-06-05 00:00:00 DEXJPUS
update 144.87 2025-06-06 00:00:00 DEXJPUS
update 1.1432 2025-06-02 00:00:00 DEXUSEU
update 1.1373 2025-06-03 00:00:00 DEXUSEU
update 1.1424 2025-06-04 00:00:00 DEXUSEU
update 1.144 2025-06-05 00:00:00 DEXUSEU
update 1.1397 2025-06-06 00:00:00 DEXUSEU
update 4.46 2025-06-02 00:00:00 DGS10
update 4.46 2025-06-03 00:00:00 DGS10
update 4.37 2025-06-04 00:00:00 DG

In [57]:
conn.close()